In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import urllib
import zipfile
import os
from scipy.fft import rfft, rfftfreq

In [2]:
ORIGINAL_SAMPLE_RATE = 50000 # taxa de amostragem original dos dados
target_rate = 500

In [3]:
def apply_rfft(signal, sampling_rate):
    """
    Aplica a Transformada Rápida de Fourier Real (RFFT) a um sinal de entrada.

    Parâmetros:
    signal (numpy array): O sinal de entrada.
    sampling_rate (float): A taxa de amostragem do sinal.

    Retorna:
    tuple: Frequências e coeficientes da RFFT.
    """
    # Aplicar RFFT
    rfft_result = np.abs(np.fft.rfft(signal, axis=0))[1:,:] #/ signal.shape[0]
    rfft_freqs = np.fft.rfftfreq(len(signal), d=1/sampling_rate)[1:]

    # spectrum = np.fft.rfftn(data)

#     # Calcular as frequências correspondentes
#     freqs = np.fft.rfftfreq(len(data), d=1/sample_rate)
#     # freqs = [np.fft.fftfreq(n, d=1/sample_rate) for n in data.shape]

    return rfft_result, rfft_freqs

In [27]:
def resample(data, sample_target_rate, qtt_series=1):
    # data_resampled = pd.DataFrame()
    data_resampled = []
    period = 5 * qtt_series # seconds
    step = int(len(data)/period / sample_target_rate )
    start_index = 0
    stop_index = step
    for i in range(0, len(data), step):
        # data_resampled = pd.concat([data_resampled, data[start_index:stop_index].mean().to_frame().T], axis=0, ignore_index=True) ## para funcionar com dataframe
        # temp_df =  pd.DataFrame([data[start_index:stop_index].mean(axis=0)])
        temp = data[start_index:stop_index].mean(axis=0)
        # data_resampled = np.concatenate([data_resampled, temp]) # para ndarray
        data_resampled.append(temp)
        # data_resampled = pd.concat([data_resampled, temp_df], ignore_index=True)
        start_index += step
        stop_index += step

    return np.array(data_resampled)

In [28]:
def process_data(files_folder):
    # processed_data = []
    data_resampled = np.empty((0, 8))
    # data = dataReader(files_folder)
    # for i in range(len(data)):
    #     data_resampled = resample(data[i], target_rate)
    #     spectrum, xf = apply_fft(data_resampled, target_rate)
    #     processed_data.append(spectrum)

    # data = dataReader2d(files_folder)
    # dataScaled = scalerData(data)

    for i in range(len(files_folder)):
      data = pd.read_csv(files_folder[i], header=None)
      data = data.to_numpy();
      file_resampled = resample(data, target_rate)
      # data_resampled = np.concatenate(data_resampled, file_resampled)
      data_resampled.append(file_resampled)
        
    spectrum, xf = apply_rfft(data_resampled, target_rate)
    # data_resampled = resample(data, target_rate, len(files_folder))
    # spectrum, xf = apply_rfft(data_resampled, target_rate)
    # processed_data.append(spectrum)

    return spectrum

In [31]:
data_normal = process_data( glob.glob('./data/normal/*.csv'))

KeyboardInterrupt: 

In [29]:
import numpy as np 


param_dist = {
    'C': [1, 10, 100],
    'gamma': [1, 0.1, 0.001, 'scale'],
    'kernel': ['linear', 'rbf', 'poly']
}

print({key: (dist.rvs() if hasattr(dist, 'rvs') else np.random.choice(dist)) if key != 'gamma' else (dist if dist == 'scale' else float(np.random.choice(dist))) for key, dist in param_dist.items()})
# print({key: dist.rvs() if hasattr(dist, 'rvs') else (np.random.choice(list(dist)) if key != 'gamma' else dist) for key, dist in param_dist.items()})

ValueError: could not convert string to float: 'scale'